In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

import numpy as np
import pandas as pd

In [0]:
data = pd.read_csv("data/Sentiment.csv")

data.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [0]:
X = list(data['SentimentText'])
Y = np.array(data[['Sentiment']])

X = [i.strip() for i in X]

X = np.array(X)

In [0]:
t = Tokenizer(20000)
t.fit_on_texts(X)

In [0]:
import pickle

with open('token' , "wb") as file:
  pickle.dump(t , file)

In [0]:
X = tf.keras.preprocessing.sequence.pad_sequences(t.texts_to_sequences(X) , 500)

In [0]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(20000 , 32 , input_length = 500))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(100))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(50))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1 , activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
his = model.fit(X,Y, epochs = 5)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
99989/99989 [==============================] - 129s 1ms/sample - loss: 0.5622 - acc: 0.7101
Epoch 2/5
99989/99989 [==============================] - 127s 1ms/sample - loss: 0.4603 - acc: 0.7940
Epoch 3/5
99989/99989 [==============================] - 126s 1ms/sample - loss: 0.3984 - acc: 0.8252
Epoch 4/5
99989/99989 [==============================] - 126s 1ms/sample - loss: 0.3196 - acc: 0.8630
Epoch 5/5
99989/99989 [==============================] - 126s 1ms/sample - loss: 0.2397 - acc: 0.9009


In [0]:
text = ["That Is Great"]

word = pad_sequences(t.texts_to_sequences(text) , 500)
print(model.predict(np.array(word)))


[[0.98566145]]


In [0]:
model.save_weights('sentiment_final.h5')